In [5]:
import numpy as np
import cv2
import sys
import glob
from matplotlib import pyplot as plt

In [3]:
def get_HOG_descriptors(img_gray, kps, win_size=(64, 64)):
    result = []
    for i, kp in enumerate(kps):
        # Read (x,y) coordinate from the current keypoint
        # https://docs.opencv.org/3.2.0/d2/d29/classcv_1_1KeyPoint.html
        x, y = kp.pt[0], kp.pt[1]

        # Create a win_size crop window for this keypoint
        left = int(x - (win_size[0]//2))
        right = int(x + (win_size[0]//2))
        if left < 0:
            left = 0
            right = left + win_size[0] + 1
        if right > img_gray.shape[1]-1:
            right = img_gray.shape[1]
            left = right - win_size[0] - 1
        top = int(y - (win_size[1]//2))
        bottom = int(y + (win_size[1]//2))
        if top < 0:
            top = 0
            bottom = top + win_size[1] + 1
        if bottom > img_gray.shape[0]-1:
            bottom = img_gray.shape[0]
            top = bottom - win_size[1] - 1

        # Crop the image with the window
        img_crop = img_gray[top:bottom, left:right]

        # Compute a HOG descriptor
        # Be careful, dimension of img_crop must match with winSize (as specified at hog initiation)
        desc = hog.compute(img_crop)

        # Add the descriptor to the resultant list
        result.append(desc)

    return np.asarray(result, np.float32)


In [4]:
# Initiate FAST detector
fast = cv2.FastFeatureDetector_create()

# Initiate HOG descriptor with default parameters
# size of the image sent to compute HOG
winSize = (16, 16)
# block size for block normalization
blockSize = tuple([i//2 for i in winSize])
blockStride = tuple([i//2 for i in blockSize])
# cell size (one cell for one 9-bin HOG)
cellSize = blockSize
nbins = 9
derivAperture = 1
winSigma = 4.
histogramNormType = 0
L2HysThreshold = 2.0000000000000001e-01
gammaCorrection = 0
nlevels = 64
hog = cv2.HOGDescriptor(winSize, blockSize, blockStride, cellSize, nbins, derivAperture, winSigma,
                        histogramNormType, L2HysThreshold, gammaCorrection, nlevels)

# Initiate FLANN matching
# Since HOG descriptor is similar to SIFT/SURF descriptor, use the same matching technique
FLANN_INDEX_KDTREE = 0
index_params = dict(algorithm=FLANN_INDEX_KDTREE, trees=5)
search_params = dict(checks=50)
flann = cv2.FlannBasedMatcher(index_params, search_params)


In [18]:
# training data
dataset_path = "dataset/image/design1/*.jpg"
files = glob.glob(dataset_path) 

In [24]:
# create template
template_name = []
template_desc = []
for item in files:
    name = item.split('/')[-1].split('.')[0]
    template_name.append(name)
    template_bgr = cv2.imread(item)
    template_gray = cv2.cvtColor(template_bgr, cv2.COLOR_BGR2GRAY)
    template_kps = fast.detect(template_gray, None)
    template_desc.append(get_HOG_descriptors(template_gray, template_kps, winSize))

In [25]:
# test data
test_path = "dataset/image/real/*.png"
test_files = glob.glob(test_path) 

[array([[[0.01037906],
         [0.00427937],
         [0.57559925],
         ...,
         [0.00144991],
         [0.00189758],
         [0.00499074]],
 
        [[0.00574996],
         [0.00368281],
         [0.01235477],
         ...,
         [0.596878  ],
         [0.02247768],
         [0.00267908]],
 
        [[0.00690606],
         [0.00144184],
         [0.57553905],
         ...,
         [0.00729835],
         [0.00844833],
         [0.02668325]],
 
        ...,
 
        [[0.33373645],
         [0.32500067],
         [0.33373645],
         ...,
         [0.31535718],
         [0.14768083],
         [0.3993264 ]],
 
        [[0.00496625],
         [0.00606   ],
         [0.00115162],
         ...,
         [0.22483173],
         [0.00477791],
         [0.01056349]],
 
        [[0.01624949],
         [0.01540361],
         [0.37775758],
         ...,
         [0.00996355],
         [0.00563794],
         [0.01856442]]], dtype=float32),
 array([[[0.01006412],
         [0.00638